# [Build a PDF ingestion and Question/Answering system](https://python.langchain.com/docs/tutorials/pdf_qa/)


In [26]:
# PDFのダウンロード (BEM280のデータシート)
!mkdir -p langchain-tutorial/resources
!wget https://www.mouser.jp/datasheet/2/783/bst_bme280_ds002-2238172.pdf -O langchain-tutorial/resources/bme280_datasheet.pdf > /dev/null 2>&1

# ■ プレビュー

In [28]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# モデル
llm = ChatOpenAI(model="gpt-4o")

# PDFの読み込み
file_path = "langchain-tutorial/resources/bme280_datasheet.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

# 分割 (1000文字ごとに分割)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# ベクトル化して保存
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieverの作成
retriever = vectorstore.as_retriever()

# チェーンの作成
system_prompt = """
あなたは質問応答のアシスタントです。
質問に答えるために、検索された文脈の以下の部分を使用してください。
答えがわからない場合は、わからないと答えましょう。

{context}
"""

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=question_answer_chain)

In [29]:
results = rag_chain.invoke({"input": "データの送受信の仕様を教えて"})
results

{'input': 'データの送受信の仕様を教えて',
 'context': [Document(id='b553b761-7a40-4958-84a5-e8a177df5d31', metadata={'source': 'langchain-tutorial/resources/bme280_datasheet.pdf', 'page': 6}, page_content='10.3 Usage ...................................................................................................................................... 55 \n10.3.1 File and function pointer integration ....................................................................... 55 \n10.3.2 Function call ............................................................................................................ 55 \n10.3.3 Test time and interface requirements ..................................................................... 55 \n10.4 Function explanation ............................................................................................................... 56 \n10.4.1 Communication test ................................................................................................ 56 \n10.4.2 Bon

# ■ プレビュー (エージェント)

In [39]:
from typing import List
from langchain_core.documents import Document
from langgraph.graph.graph import CompiledGraph

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools.retriever import create_retriever_tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# モデル
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# PDFの読み込み
file_path = "langchain-tutorial/resources/bme280_datasheet.pdf"
loader = PyPDFLoader(file_path)
docs: List[Document] = loader.load()

# 分割・ベクトル化して保存
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits: List[Document] = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieverの作成
retriever = vectorstore.as_retriever()

# Retrieverツールの作成
tool = create_retriever_tool(
    retriever=retriever,
    name="bme280_datasheet_retriever",
    description="BME280のデータシートから情報を検索するためのツール",
)

# エージェントの作成
memory = MemorySaver()
agent_executor: CompiledGraph = create_react_agent(
    model=llm,
    tools=[tool],
    checkpointer=memory,
    debug=False,
)

In [40]:
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage

config = RunnableConfig(configurable={"thread_id": "11111111111111111111"})
query = "データの送受信の仕様を教えて"

output_stream = agent_executor.stream(
    input={"messages": [ HumanMessage(query) ]},
    config=config,
    stream_mode="values"
)

for event in output_stream:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

データの送受信の仕様を教えて
================================== Ai Message ==================================
Tool Calls:
  bme280_datasheet_retriever (call_jrjoQRQsorF6ID2AKolFNUPf)
 Call ID: call_jrjoQRQsorF6ID2AKolFNUPf
  Args:
    query: データの送受信の仕様
================================= Tool Message =================================
Name: bme280_datasheet_retriever

10.3 Usage ...................................................................................................................................... 55 
10.3.1 File and function pointer integration ....................................................................... 55 
10.3.2 Function call ............................................................................................................ 55 
10.3.3 Test time and interface requirements ..................................................................... 55 
10.4 Function explanation ...............

In [41]:
query = "SPIインターフェースについて教えて"

output_stream = agent_executor.stream(
    input={"messages": [ HumanMessage(query) ]},
    config=config,
    stream_mode="values"
)

for event in output_stream:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

SPIインターフェースについて教えて
================================== Ai Message ==================================
Tool Calls:
  bme280_datasheet_retriever (call_ieoga2Wi2TUf2kwBbLNDhXyR)
 Call ID: call_ieoga2Wi2TUf2kwBbLNDhXyR
  Args:
    query: SPI interface
================================= Tool Message =================================
Name: bme280_datasheet_retriever

6.3.1 SPI write.................................................................................................................. 34 
6.3.2 SPI read .................................................................................................................. 35 
6.4 Interface parameter specification ............................................................................................ 35 
6.4.1 General interface parameters ................................................................................. 35 
6.4.2 I²C timings ...............

# ■ 解説

## 1. PDFの読み込み

`PyPDFLoader` を利用してPDFを読み込みます。  

※ 目的に合わせて様々なDocumentLoaderがあります: https://python.langchain.com/docs/how_to/#document-loaders

やること

- 指定されたパスのPDFをメモリに読み込む
- `pypdf` パッケージを利用してテキストデータを抽出
- PDFの各ページに対して、ページ内容と、ドキュメント内のテキストの出所に関するメタデータを含む `List[Document]` を生成

In [30]:
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

#file_path = "https://www.jstage.jst.go.jp/article/mukimate2000/9/300/9_300_327/_pdf"  # URLを指定してもOK
file_path = "langchain-tutorial/resources/bme280_datasheet.pdf"
loader = PyPDFLoader(file_path)
docs: List[Document] = loader.load()

print(len(docs))

60


In [31]:
print(docs[0].page_content[0:100])
print(docs[3].metadata)

 
  
  
BME280 – Data sheet  
Document revision 1.22 
Document release date October 2021 
Document n
{'source': 'langchain-tutorial/resources/bme280_datasheet.pdf', 'page': 3}


## 2. Retriever

読み込んだドキュメントをベクトル化して保存します。

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model="gpt-4o")

# 分割 (1000文字ごとに分割)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits: List[Document] = text_splitter.split_documents(docs)

# ベクトル化して保存
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieverの作成
retriever = vectorstore.as_retriever()

vectorstoreに保存したドキュメントをRetrieverから検索できます

In [33]:
retriever.invoke("測定レンジと精度")

[Document(id='50bd48e3-bac6-4bd2-aa75-a280e6e5735d', metadata={'source': 'langchain-tutorial/resources/bme280_datasheet.pdf', 'page': 10}, page_content='Solder drift  Minimum solder height \n50µm \n-0.5  +2.0 hPa \nLong term stability6 ∆Pstab per year  ±1.0  hPa \nPossible sampling rate fsample_P Lowest oversampling, \nsee chapter 9.2 \n157 182  Hz \n \n1.4 Temperature sensor specification \nTable 4: Temperature parameter specification \nParameter Symbol Condition Min Typ Max Unit \nOperating range T Operational -40 25 85 °C \nFull accuracy 0  65 °C \nSupply current IDD,T 1 Hz forced mode, \ntemperature \nmeasurement only \n 1.0  µA \nAT,25 25 °C  ±0.5  °C \n                                                      \n5 When changing temperature by e.g. 10 °C at constant pressure / altitude, the measured pressure / altitude will change by (10 × \nTCOP).  \n6 Long term stability is specified in the full accuracy operating pressure range 0 … 65 °C'),
 Document(id='3a124578-139b-41fe-b779-5315

# RAGチェイン

`クエリ` -> `Retrieverによる関連文章の検索` -> `LLMによる回答の生成` というチェインを作成します。


In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_prompt = """
あなたは質問応答のアシスタントです。
質問に答えるために、検索された文脈の以下の部分を使用してください。
答えがわからない場合は、わからないと答えましょう。

{context}
"""

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=question_answer_chain)

In [35]:
results = rag_chain.invoke({"input": "温度の測定レンジと精度を教えて"})
results

{'input': '温度の測定レンジと精度を教えて',
 'context': [Document(id='29a0a41b-eb29-4843-a4fb-1493f457d883', metadata={'source': 'langchain-tutorial/resources/bme280_datasheet.pdf', 'page': 11}, page_content='Bosch Sensortec | BME280 Data sheet 12 | 60 \nModifications reserved | Data subject to change without notice  \n Document number: BST-BME280-DS001-22 Revision_1.22_102021 \nAbsolute accuracy \ntemperature7 \nAT,full 0…65 °C  ±0.5  °C \nAT,ext8 -20 …. 0 °C  ±1.25  °C \nAT,ext9 -40 … -20 °C  ±1.5  °C \nOutput resolution RT API output resolution 0.01 °C \nRMS noise NT Lowest oversampling  0.005  °C \n \n  \n                                                      \n7 Temperature measured by the internal temperature sensor. This temperature value depends on the PCB temperature, sensor \nelement self-heating and ambient temperature and is typically above ambient temperature.  \n8 Target values & not guaranteed \n9 Target values & not guaranteed'),
  Document(id='50bd48e3-bac6-4bd2-aa75-a280e6e5735d', me

In [36]:
results = rag_chain.invoke({"input": "データの送受信の仕様を教えて"})
results

{'input': 'データの送受信の仕様を教えて',
 'context': [Document(id='e20387c4-1bcf-4d69-b58d-969cc70deaa6', metadata={'source': 'langchain-tutorial/resources/bme280_datasheet.pdf', 'page': 6}, page_content='10.3 Usage ...................................................................................................................................... 55 \n10.3.1 File and function pointer integration ....................................................................... 55 \n10.3.2 Function call ............................................................................................................ 55 \n10.3.3 Test time and interface requirements ..................................................................... 55 \n10.4 Function explanation ............................................................................................................... 56 \n10.4.1 Communication test ................................................................................................ 56 \n10.4.2 Bon

In [37]:
print(results["answer"])

データの送受信に関する仕様は、SPIプロトコルに基づいています。以下がその概要です：

- CSB（チップセレクトビット）はアクティブローで、統合されたプルアップ抵抗があります。
- SDI（シリアルデータ入力）のデータは、SCK（シリアルクロック）の立ち上がりエッジでデバイスによってラッチされます。
- SDO（シリアルデータ出力）は、SCKの立ち下がりエッジで変更されます。
- 通信はCSBがローになったときに開始し、ハイになったときに停止します。このCSBの遷移中は、SCKが安定している必要があります。
- SPIモードでは、レジスタアドレスの上位7ビットのみが使用され、MSBは使用されず、代わりに読み書きビット（RW：書き込みの場合は‘0’、読み取りの場合は‘1’）に置き換えられます。

具体例として、アドレス0xF7にアクセスする場合、SPIレジスタアドレス0x77を使用します。書き込みアクセスの場合はバイト0x77が転送され、読み取りアクセスの場合はバイト0xF7が転送されます。
